In [2]:
import time
import datetime
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support import expected_conditions as EC

from concurrent.futures import ThreadPoolExecutor
import multiprocessing

from Functions.function import movies_for_category, categories, print_process, movies
from bs4 import BeautifulSoup
import requests
import re


In [7]:
print_process("Comienzo del Script")

|-----------------------------Comienzo del Script-----------------------------|


In [8]:
categorias = categories()

|---------------------Categorias capturadas correctamente.---------------------|
|-----------------------Categorias demoro 8.79 segundos-----------------------|


In [3]:

# Obtener la cantidad de núcleos disponibles en el sistema
num_nucleos = multiprocessing.cpu_count()

# Crear un diccionario para almacenar los resultados
resultados_dict = {}

categorias_comienzo = time.time()

with ThreadPoolExecutor(max_workers=num_nucleos) as executor:
    # Ejecutar la función para cada categoría en paralelo
    resultados = list(executor.map(movies_for_category, categorias))
    
    # Asignar los resultados al diccionario
    for categoria, resultado in zip(categorias, resultados):
        resultados_dict[categoria] = resultado

peliculas_total = []

for x,i in resultados_dict.items():
    for e in i:
        peliculas_total.append(e)

peliculas_total = set(peliculas_total)

categorias_fin = time.time()

print_process(f"La extraccion total de las categorias tubo una demora de {round((categorias_fin-categorias_comienzo),2)} segundos")


|Todo el contenido de la categoria 'biography' fue capturado correctamente, demoro 21.38 segundos|
|Todo el contenido de la categoria 'history' fue capturado correctamente, demoro 22.01 segundos|
|Todo el contenido de la categoria 'fantasy' fue capturado correctamente, demoro 23.08 segundos|
|Todo el contenido de la categoria 'adventure' fue capturado correctamente, demoro 23.23 segundos|
|Todo el contenido de la categoria 'family' fue capturado correctamente, demoro 24.04 segundos|
|Todo el contenido de la categoria 'animation' fue capturado correctamente, demoro 25.36 segundos|
|Todo el contenido de la categoria 'crime' fue capturado correctamente, demoro 25.39 segundos|
|Todo el contenido de la categoria 'action' fue capturado correctamente, demoro 27.38 segundos|
|Todo el contenido de la categoria 'horror' fue capturado correctamente, demoro 36.1 segundos|
|Todo el contenido de la categoria 'musical' fue capturado correctamente, demoro 9.4 segundos|
|Todo el contenido de la categor

In [4]:
peliculas_total = [x for x in peliculas_total if "/content/" in x]

In [5]:
# Crear un diccionario para almacenar los resultados
todo = {"titulo": [],"idioma" : [], "subtitulos" : [], "ano": [], "duracion_minutos": [], "categorias": [], "sinopsis": [], "tipo": [], "link": []}

comienzo_movies = time.time()

with ThreadPoolExecutor(max_workers=num_nucleos) as executor:
    # Ejecutar la función para cada categoría en paralelo
    resultados = list(executor.map(movies, peliculas_total))

    try:
        for resultado in resultados:
            for clave, valor in resultado.items():
                todo[clave].append(valor)
    except:
        print()

fin_movies = time.time()

print_process(f"El proceso de captura de la informacion de todo el contenido fue exsitoso, y demoro {round((fin_movies - comienzo_movies),2)} segundos")


Error al cargar la url https://filmzie.com/content/autumn-moon-2023 en el intento 1: 404
|El proceso de captura de la informacion de todo el contenido fue exsitoso, y demoro 362.18 segundos|


|------------------------------------------------------------------------------|


In [6]:
df = pd.DataFrame(todo)

In [23]:
df = pd.read_csv("Bases_datos/Peliculas_Series_Completo_Respaldo.csv")

In [24]:
# df.to_csv("Bases_datos/Peliculas_Series_Completo_Respaldo.csv" , index=False)

In [25]:
peliculas = df.loc[df["tipo"] == "Pelicula"].reset_index(drop=True)
series = df.loc[df["tipo"] == "Serie"].reset_index(drop=True)

In [26]:
# Limpieza de metadata peliculas

peliculas['categorias'] = peliculas['categorias'].str.split(', ')
all_categories = set([cat for sublist in peliculas['categorias'].tolist() for cat in sublist])

for category in all_categories:
    peliculas[category] = 0
    for index, row in peliculas.iterrows():
        if category in row['categorias']:
            peliculas.at[index, category] = 1

peliculas.drop(columns=['categorias'], inplace=True)

peliculas["duracion_minutos"] = peliculas["duracion_minutos"].str.replace(" min","").astype(int)



In [37]:
peliculas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2204 entries, 0 to 2203
Data columns (total 33 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   titulo               2204 non-null   object
 1   idioma               2204 non-null   object
 2   subtitulos           1638 non-null   object
 3   ano                  2204 non-null   int64 
 4   duracion_minutos     2204 non-null   int32 
 5   sinopsis             2204 non-null   object
 6   tipo                 2204 non-null   object
 7   link                 2204 non-null   object
 8   European Production  2204 non-null   int64 
 9   Western              2204 non-null   int64 
 10  Biography            2204 non-null   int64 
 11  Thriller             2204 non-null   int64 
 12  Comedy               2204 non-null   int64 
 13  Romance              2204 non-null   int64 
 14  Reality TV           2204 non-null   int64 
 15  Sport                2204 non-null   int64 
 16  Music 

In [38]:
peliculas

,titulo,idioma,subtitulos,ano,duracion_minutos,sinopsis,tipo,link,European Production,Western,...,Horror,Adventure,Crime,Family,Fantasy,Drama,News,Mystery,Animation,Documentary
0,Play Ball,Spanish,"English, Portuguese",2008,96,A promising MLB player is forced to return to ...,Pelicula,https://filmzie.com/content/play-ball-2008,0,0,...,0,0,0,0,0,1,0,0,0,0
1,The Musical Adventures of Afro Banzai,English,"English, Spanish, French, Hindi, Japanese, Ukr...",2021,55,A group of culturally diverse teens who each r...,Pelicula,https://filmzie.com/content/the-musical-advent...,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Waste: The Nuclear Nightmare,English,NaN,2009,77,"Explore the complexities of nuclear waste in ""...",Pelicula,https://filmzie.com/content/waste-the-nuclear-...,0,0,...,0,0,0,0,0,0,0,0,0,1
3,Atmosphere,Spanish,English (embedded),2010,20,"Cecilia, Alberto, and Felipe face an epidemic ...",Pelicula,https://filmzie.com/content/atmosphere,0,0,...,0,0,0,0,0,1,0,0,0,0
4,Theresa + Allison,English,English,2019,120,Theresa's life takes a dark turn after a disas...,Pelicula,https://filmzie.com/content/theresa--allison-2019,0,0,...,1,0,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2199,Jennifer Aniston: More Than Friends,English,English,2020,58,Explore the multifaceted journey of global sup...,Pelicula,https://filmzie.com/content/jennifer-aniston-m...,0,0,...,0,0,0,0,0,0,0,0,0,1
2200,No Time for Quiet,English,English,2017,80,"Over a long hot summer, 40 girls and gender di...",Pelicula,https://filmzie.com/content/no-time-for-quiet-...,0,0,...,0,0,0,0,0,0,0,0,0,1
2201,Sara,Spanish,English,2017,25,"In memory of Ramiro Tejada, experience a Carol...",Pelicula,https://filmzie.com/content/sara-2017,0,0,...,0,0,0,0,0,0,0,0,0,0
2202,Lies of Omission,English,NaN,2017,65,Embrace the uncomfortable truth. Amid a world ...,Pelicula,https://filmzie.com/content/lies-of-omission-2017,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
peliculas.to_csv("Bases_datos/Peliculas.csv",index=False)

In [ ]:
df.loc[df["tipo"] == "Serie"]